In [28]:
import numpy as np

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from tqdm import tqdm

import time

from scipy.interpolate import interp1d
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [29]:
# Число частиц
N = 60
N_values = np.arange(int(N - 0.05 * N), int(N + 0.05 * N)+1)
# Размер двумерной области
x_size = 10
y_size = 10
# Масса одной частицы
m = 0.2
sigma = 0.03
epsilon = 0.05
r_min = 0.028
kBT = 0.1
# Число степеней свободы для частицы
f = 2
# Постоянная Больцмана
kB = 1
T = kBT/kB
T_values = np.round(np.linspace((kBT - 0.1 * kBT), (kBT + 0.1 * kBT), 3), 2)

In [30]:
print(N_values)

[57 58 59 60 61 62 63]


In [31]:
print(T_values)

[0.09 0.1  0.11]


In [32]:
def modified_potential_energy(positions, r_end, forces, n):
    energy = 0.0
    energy_end = 4 * epsilon * ((sigma / r_end)**12 - (sigma / r_end)**6)
    for i in range(n):
        for j in range(i + 1, n):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:  
                energy += (((4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6)) + np.dot(forces[i], forces[j])*(r_min-r)) - energy_end) # Аппрокисмация
            elif r < r_end:
                energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
            else:
                continue
        # Левая стенка  
        if positions[i, 0] < r_end and positions[i, 0] > r_min:
            r = positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] < r_end and positions[i, 0] < r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
        # Правая стенка   
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min:
            r = x_size - positions[i, 0]
            energy += (4 * epsilon * ((sigma / r)**12 - (sigma / r)**6) - energy_end)
        elif positions[i, 0] > x_size - r_end and positions[i, 0] > x_size - r_min:
            energy += (4 * epsilon * ((sigma / r_min)**12 - (sigma / r_min)**6) + (4 * epsilon * ((12 * (sigma / r_min)**12) - (6 * (sigma / r_min)**6))*(r_min-r)) - energy_end)
        else:
            continue
    return energy

In [33]:
def modified_forces_calc(positions, r_end, n):
    force = np.zeros_like(positions)
    virial = 0.0  

    for i in range(n):
        for j in range(i + 1, n):
            r_vec = positions[i] - positions[j]
            r_vec[1] -= y_size * np.round(r_vec[1] / y_size)  # Периодические условия по оси y
            r = np.linalg.norm(r_vec)
            if r < r_min:
                f_mag = 4 * epsilon * (12 * (sigma / r_min)**12 - 6 * (sigma / r_min)**6)
            elif r < r_end:
                f_mag = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            else:
                continue

            f_vec = f_mag * (r_vec / r)
            force[i] += f_vec
            force[j] -= f_vec

            virial += np.dot(r_vec, f_vec)

        f_mag_x = 0.0
        # Левая стенка
        if positions[i, 0] < r_end and positions[i, 0] > r_min:
            r = positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x  # r_x * F_x

        elif positions[i, 0] < r_end and positions[i, 0] <= r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] += f_mag_x
            virial += positions[i, 0] * f_mag_x

        # Правая стенка
        elif positions[i, 0] > x_size - r_end and positions[i, 0] < x_size - r_min:
            r = x_size - positions[i, 0]
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)  # r_x * F_x

        elif positions[i, 0] >= x_size - r_min:
            r = r_min
            f_mag_x = 4 * epsilon * (12 * (sigma / r)**12 - 6 * (sigma / r)**6)
            force[i, 0] -= f_mag_x
            virial += (positions[i, 0] - x_size) * (-f_mag_x)

    return force, virial

In [36]:
pressures = []
dt = 0.1
r_end = 2.5 * sigma
timesteps = np.arange(0, 1001, dt)
#velocities_4 = velocities.copy()
#pos_4 = pos.copy()
V = x_size * y_size
trajectory = []


start_time = time.time()
for n in N_values: # Частицы в диапазоне, а температуры фиксированные 
    instant_pressures = []
    
    n_side_x = int(np.ceil(np.sqrt(n * x_size / y_size)))
    n_side_y = int(np.ceil(np.sqrt(n * y_size / x_size)))
    spacing_x = x_size / n_side_x
    spacing_y = y_size / n_side_y
    pos = []
    for i in range(n_side_x):
        for j in range(n_side_y):
            if len(pos) < n:
                x = i * spacing_x + spacing_x/2
                y = j * spacing_y + spacing_y/2
                pos.append([x, y])
    pos = np.array(pos)          
    velocities = np.random.normal(0, np.sqrt(T / m), (n, 2))
    velocities -= np.mean(velocities, axis=0)
                   
    pos_copy = pos.copy()
    
    E_kin = np.sum(0.5 * m * velocities**2)
    temperature = (1 / (n * f * kB)) * E_kin
    for t in timesteps:
            
        forces, virial = modified_forces_calc(pos_copy, r_end, n)
        a = forces / m
        velocities_i = velocities + a * dt
            
            # Термостат Берендсена
        scale = np.sqrt(1 + (dt / 0.1) * (T / temperature - 1))
        velocities_i *= scale
            
        pos_i = pos_copy + ((velocities + velocities_i) / 2) * dt
            # Периодические условия по оси y
        pos_i[:, 1] %= y_size
            # Искусственные стенки по оси x
        pos_i[:, 0] = np.clip(pos_i[:, 0], 0.01, x_size-0.01)
        E_kin = np.sum(0.5 * m * velocities_i**2)
        temperature = (1 / (n * f * kB)) * E_kin
        if t % 100 == 0:
            pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
            instant_pressures.append(pressure)
        velocities = velocities_i
        pos_copy = pos_i
    pressures.append(np.mean(instant_pressures))
    
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')

pressures = np.array(pressures)
std_pressure = np.std(pressures)
print('Неопределенность выходной величины(давления): '+str(std_pressure))

N_values_sorted = np.sort(N_values)

y_P_N = pressures 
X_N = N_values_sorted.reshape(-1, 1)

model_N = LinearRegression().fit(X_N, y_P_N)

dP_dN = model_N.coef_[1]
print('Производная чувствительности dP/dN: ', dP_dN)

Время расчета:  1132.1184957027435  секунд
Неопределенность выходной величины(давления): 0.0030706174738344043
Производная чувствительности dP/dN:  0.0014173409633745774


In [37]:
pressures = []
dt = 0.1
r_end = 2.5 * sigma
timesteps = np.arange(0, 1001, dt)
#velocities_4 = velocities.copy()
#pos_4 = pos.copy()
V = x_size * y_size
trajectory = []

start_time = time.time()
n_side_x = int(np.ceil(np.sqrt(N * x_size / y_size)))
n_side_y = int(np.ceil(np.sqrt(N * y_size / x_size)))
spacing_x = x_size / n_side_x
spacing_y = y_size / n_side_y
pos = []
for i in range(n_side_x):
    for j in range(n_side_y):
        if len(pos) < N:
            x = i * spacing_x + spacing_x/2
            y = j * spacing_y + spacing_y/2
            pos.append([x, y])
pos = np.array(pos)
for tetta in T_values:
    instant_pressures = []
    velocities = np.random.normal(0, np.sqrt(tetta / m), (N, 2))
    velocities -= np.mean(velocities, axis=0)
    pos_copy = pos.copy()
    E_kin = np.sum(0.5 * m * velocities**2)
    temperature = (1 / (N * f * kB)) * E_kin
    for t in timesteps:
            
        forces, virial = modified_forces_calc(pos_copy, r_end, N)
        a = forces / m
        velocities_i = velocities + a * dt
            
        # Термостат Берендсена
        scale = np.sqrt(1 + (dt / 0.1) * (tetta / temperature - 1))
        velocities_i *= scale
            
        pos_i = pos_copy + ((velocities + velocities_i) / 2) * dt
            # Периодические условия по оси y
        pos_i[:, 1] %= y_size
            # Искусственные стенки по оси x
        pos_i[:, 0] = np.clip(pos_i[:, 0], 0.01, x_size-0.01)
        E_kin = np.sum(0.5 * m * velocities_i**2)
        temperature = (1 / (n * f * kB)) * E_kin
        if t % 100 == 0:
            pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
            instant_pressures.append(pressure)
        velocities = velocities_i
        pos_copy = pos_i
    pressures.append(np.mean(instant_pressures))
    
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')

pressures = np.array(pressures)
std_pressure = np.std(pressures)
print('Неопределенность выходной величины(давления): '+str(std_pressure))

T_values_sorted = np.sort(T_values)

y_P_T = pressures 
X_T = T_values_sorted.reshape(-1, 1)

model_T = LinearRegression().fit(X_T, y_P_T)

dP_dT = model_T.coef_[1]

print('Производная чувствительности dP/dT: ', dP_dT)


Время расчета:  458.1808109283447  секунд
Неопределенность выходной величины(давления): 0.007451159969757241
Производная чувствительности dP/dT:  0.9125693473358009


In [41]:
pressures = []
dt = 0.1
r_end = 2.5 * sigma
timesteps = np.arange(0, 1001, dt)
#velocities_4 = velocities.copy()
#pos_4 = pos.copy()
V = x_size * y_size
trajectory = []

start_time = time.time()
for n in N_values:
    n_side_x = int(np.ceil(np.sqrt(n * x_size / y_size)))
    n_side_y = int(np.ceil(np.sqrt(n * y_size / x_size)))
    spacing_x = x_size / n_side_x
    spacing_y = y_size / n_side_y
    pos = []
    for i in range(n_side_x):
        for j in range(n_side_y):
            if len(pos) < n:
                x = i * spacing_x + spacing_x/2
                y = j * spacing_y + spacing_y/2
                pos.append([x, y])
    pos = np.array(pos)
    for tetta in T_values:
        instant_pressures = []
        velocities = np.random.normal(0, np.sqrt(tetta / m), (n, 2))
        velocities -= np.mean(velocities, axis=0)
        pos_copy = pos.copy()
        E_kin = np.sum(0.5 * m * velocities**2)
        temperature = (1 / (n * f * kB)) * E_kin
        for t in timesteps:
            
            forces, virial = modified_forces_calc(pos_copy, r_end, n)
            a = forces / m
            velocities_i = velocities + a * dt
            
            # Термостат Берендсена
            scale = np.sqrt(1 + (dt / 0.1) * (tetta / temperature - 1))
            velocities_i *= scale
            
            pos_i = pos_copy + ((velocities + velocities_i) / 2) * dt
            # Периодические условия по оси y
            pos_i[:, 1] %= y_size
            # Искусственные стенки по оси x
            pos_i[:, 0] = np.clip(pos_i[:, 0], 0.01, x_size-0.01)
            E_kin = np.sum(0.5 * m * velocities_i**2)
            temperature = (1 / (n * f * kB)) * E_kin
            if t % 100 == 0:
                pressure = (1 / (V * 3)) * ((E_kin * 2) + virial)
                instant_pressures.append(pressure)
            velocities = velocities_i
            pos_copy = pos_i
        pressures.append(np.mean(instant_pressures))
end_time = time.time()
elapsed_time = end_time - start_time
print('Время расчета: ', elapsed_time, ' секунд')
pressures = np.array(pressures)

std_pressure = np.std(pressures)
print('Неопределенность выходной величины(давления): '+str(std_pressure))


N_values_sorted = np.sort(N_values)
T_values_sorted = np.sort(T_values)
pressures_reshaped = pressures.reshape(len(N_values_sorted), len(T_values_sorted))

N_grid, T_grid = np.meshgrid(N_values_sorted, T_values_sorted, indexing='ij')

N_values_flat = N_grid.flatten()
T_values_flat = T_grid.flatten()
pressures_flat = pressures_reshaped.flatten()

X_multi = np.column_stack((N_values_flat, T_values_flat))
y_multi = pressures_flat

poly_multi = PolynomialFeatures(degree=2)
X_poly_multi = poly_multi.fit_transform(X_multi)

model_multi = LinearRegression().fit(X_poly_multi, y_multi)
feature_names = poly_multi.get_feature_names_out(['N', 'T'])
coefs = model_multi.coef_
intercept = model_multi.intercept_

coef_dict = dict(zip(feature_names, coefs))
coef_dict['1'] = intercept 

print("\nКоэффициенты аппроксимирующего полинома P(N, T):")
for name, coef in coef_dict.items():
    print(f"  Коэффициент при {name}: {coef}")

# Аналитические производные P(N, T) = a_20*N^2 + a_02*T^2 + a_11*NT + a_10*N + a_01*T + a_00
# dP/dT = 2*a_02*T + a_11*N + a_01
# dP/dN = 2*a_20*N + a_11*T + a_10

# Коэффициенты полиномиальной регрессии 
a_20 = coef_dict.get('N^2', 0) 
a_02 = coef_dict.get('T^2', 0)
a_11 = coef_dict.get('N T', 0) 
a_10 = coef_dict.get('N', 0)   
a_01 = coef_dict.get('T', 0)  


dP_dT_at_point = 2 * a_02 * T + a_11 * N + a_01
dP_dN_at_point = 2 * a_20 * N + a_11 * T + a_10

print(f'\nЧастная производная чувствительности dP/dT в точке (N={N:.2f}, T={T:.2f}): {dP_dT_at_point}')
print(f'Частная производная чувствительности dP/dN в точке (N={N:.2f}, T={T:.2f}): {dP_dN_at_point}')

Время расчета:  3525.5247571468353  секунд
Неопределенность выходной величины(давления): 0.007011379130842308

Коэффициенты аппроксимирующего полинома P(N, T):
  Коэффициент при 1: 0.18321073393982604
  Коэффициент при N: -0.005828375279640389
  Коэффициент при T: -0.21146811566868456
  Коэффициент при N^2: 4.8541974468511995e-05
  Коэффициент при N T: 0.014837584083794488
  Коэффициент при T^2: 0.46672762595278305

Частная производная чувствительности dP/dT в точке (N=60.00, T=0.10): 0.7721324545495414
Частная производная чувствительности dP/dN в точке (N=60.00, T=0.10): 0.0014804200649604993
